In [1]:
from sklearn.ensemble import RandomForestClassifier
import itertools
import numpy as np
import pandas as pd

In [2]:
def split_data(features):
    """
    Receive a list of featured sentences splitted by words and split it into samples and labels.

    Parameters:
    word_features(list): List of feature words. There is an empty element between sentences in order to split each one.

    Returns:
    X_samples(list): List of feature words missing the label of the word.
    Y_labels(list): List of labels for each word.
    """

    X_samples = []
    Y_labels = []

    for feat in features:
        feat = feat.split(" ")
        Y_labels.append(feat[0])
        X_samples.append(feat[1:])

    return X_samples, Y_labels


def output_predicted_entities(Y_pred, filename):
    """
    Receives the predicted list of labels by the ML model and a filename of the detailed word features and construct predicted.txt file
    with the intrinsic details of each word, id, offsets and the predicted label.

    Parameters:
    Y_pred(list): List of list of predicted labels for each word.
    filename(str): filename to read the "feats.dat" data of the testing dataset

    Returns:

    """

    Y_pred_flatten = [el for line in Y_pred for el in line]
    detailed_word_features = open(filename).read().split("\n")[:-1]

    for label, detailed_feats in zip(Y_pred_flatten, detailed_word_features):
        _id, e1_id, e2_id = detailed_feats.split(" ")[0:3]
        if label == "null":
            interaction = "0"
        else:
            interaction = "1"
        line = [_id, e1_id, e2_id, interaction, label]
        outputfile.write("|".join(line) + "\n")
        
def evaluate(inputdir, outputfile):
    """
    Receives an input directory and the outputfile to evaluate the predicted labels with the evaluateNER.jar program.

    Parameters:
    inputdir(str):
    outputfile(str):

    Returns:

    """

    os.system("java -jar eval/evaluateDDI.jar " + inputdir + " " + outputfile)


In [3]:
import collections

In [21]:
feature_names_eq = [item.split("=")[0]for item in flatten_X_samples if "=" in item]
feature_names_eq = list(set(feature_names_eq))

In [22]:
feature_names_eq

['count_not',
 'special_w_pos_end',
 'words_between',
 'special_w',
 'special_w_pos_begin',
 'neg_word_mid',
 'num_keywords',
 'neg_word_begin',
 'neg_word_end',
 'special_w_pos_mid']

In [27]:
def transform_feature_vector_to_dataset(X_train, X_test):
    
    flatten_X_samples = list(itertools.chain(*X_train))
    feature_names_eq = [item.split("=")[0]for item in flatten_X_samples if "=" in item]
    feature_names_eq = list(set(feature_names_eq))
    feature_names_without_eq = [item for item, count in collections.Counter(flatten_X_samples).items() if count >= 50 and "=" not in item][1:]
    
    n_features = len(feature_names_eq) + len(feature_names_without_eq)
    
    new_X_train = np.zeros((len(X_train), n_features))
    new_X_test = np.zeros((len(X_test), n_features))
    
    
    for i, sample in enumerate(X_train):
        for j, feature_name in enumerate(feature_names_eq):
            for feat in sample:
                if feature_name in feat:
                    value = feat.split("=")[1]                
                    new_X_train[i,j] = value
    
    for i, sample in enumerate(X_train):
        for j, feature_name in enumerate(feature_names_without_eq, len(feature_names_eq)):
            if feature_name in sample:
                new_X_train[i,j] = 1
    
    for i, sample in enumerate(X_test):
        for j, feature_name in enumerate(feature_names_eq):
            for feat in sample:
                if feature_name in feat:
                    value = feat.split("=")[1]                
                    new_X_test[i,j] = value
    
    for i, sample in enumerate(X_test):
        for j, feature_name in enumerate(feature_names_without_eq, len(feature_names_eq)):
            if feature_name in sample:
                new_X_test[i,j] = 1
    
    return new_X_train, new_X_test

def process_feature_vectors(train_feature_vectors, test_feature_vectors):
    X_train, Y_train = split_data(train_feature_vectors)
    X_test, Y_test = split_data(test_feature_vectors)
    
    new_X_train, new_X_test = transform_feature_vector_to_dataset(X_train, X_test)
    # new_X_train, new_X_test = process_dataset_to_dataframe(new_X_train, new_X_test)
    return new_X_train, Y_train, new_X_test, Y_test

In [28]:
X_train, Y_train = split_data(train_feat_vects)
X_test, Y_test = split_data(test_feat_vects)

In [29]:
flatten_X_samples = list(itertools.chain(*X_train))
feature_names_eq = [item for item in flatten_X_samples if "=" in item]
feature_names_without_eq = [item for item, count in collections.Counter(flatten_X_samples).items() if count >= 1 and "=" not in item]
# new_X_train = np.zeros((len(X_train), len(feature_names)))
# new_X_test = np.zeros((len(X_test), len(feature_names)))

In [30]:
feature_names_without_eq

['',
 'Antagonists(nmod)->reports(conj)->loss(acl:relcl)->administered(nsubjpass)',
 'levels(compound)',
 'increased(nmod)->inhibitors(conj)->blockers(compound)',
 'amiodarone(appos)->Antihistamines(compound)',
 'Acitretin(dep)->Interferes(nmod)->effect(nmod)->preparations(amod)',
 'Antihistamines(appos)->counteract(dobj)->effect(nmod)',
 'therapies(dep)->acid(amod)',
 'affected(nmod)->co-administration(nmod)->sodium(compound)',
 'solutions(acl)->containing(nmod)->patients(acl)->receiving(dobj)->inhibitors(compound)->produce(nsubj)',
 'include(dobj)->aminoglutethimide(conj)',
 'stereoisomers(nmod)->compared(nsubjpass)',
 'Phenytoin/Phenobarbital(dep)->coadministration(nmod)->affect(nsubj)',
 'Rifabutin(dep)->Coadministration(nmod)->resulted(nsubj)',
 'coadministered(nmod)->lamivudine(conj)',
 'Drugs(dep)->administration(nmod)->drug(amod)->increase(nsubj)',
 'Inhibitors(appos)->demonstrate(ccomp)->enhanced(nmod)',
 'inhibitors(conj)->drugs(amod)',
 ',(root)->rifapentine(punct)->,(root)-

In [37]:
### MAIN VARIABLES
# train_filename = os.path.join(args.train, "megam.dat")
# test_filename = os.path.join(args.test, "megam.dat")
# fulltest_filename = os.path.join(args.test, "feats.dat")

train_filename = "./extracted_features/Train/megam.dat"
test_filename = "./extracted_features/Devel/megam.dat"
fulltest_filename = "./extracted_features/Devel/feats.dat"

# _type represents the if it has internal or external knowledge
# _type = args.test.split("/")[-2]
# inputdir = os.path.join("data", args.test.split("/")[-1])

# If outputdir does not exist, we safely create it
# outputdir = "./"
# if not os.path.exists(outputdir):
#    os.makedirs(outputdir)


output_filename = "./predicted.txt"


train_feat_vects = open(train_filename, "r").read().split("\n")[:-1]
test_feat_vects = open(test_filename, "r").read().split("\n")[:-1]

X_train, Y_train, X_test, Y_test = process_feature_vectors(train_feat_vects, test_feat_vects)

ValueError: could not convert string to float: nmod

In [32]:
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred = [[value] for value in Y_pred]

/home/ggebbia/.local/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [33]:
from sklearn.model_selection import GridSearchCV


In [34]:
def gridsearch(model, parameters):
    clf = GridSearchCV(model, parameters)
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)

In [35]:
rfc_cv = RandomForestClassifier()
params = {'n_estimators': [5, 10, 15, 20],
         'max_depth': [100, 200, 300]}
gridsearch(rfc_cv, params)

/home/ggebbia/.local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/ggebbia/.local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:651: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [36]:
outputfile = open(output_filename, "w")
output_predicted_entities(Y_pred, fulltest_filename)
outputfile.close()

In [ ]:
set 